Ensemble of the three notebooks.Please upvote the original notebooks:

**1. Deberta v3 large**
[PPPM / Deberta-v3-large baseline [inference]](https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-inference)

**2. Roberta-large**
[PatentPhrase RoBERTa Inference](https://www.kaggle.com/code/santhoshkumarv/patentphrase-roberta-inference-lb-0-814)

**3. Bert-for-papent**

And use ensemble strategy from:
[Tips for ensambling](https://www.kaggle.com/code/jellyz9/tips-for-ensambling)

## 1.Deberta v3 large

In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [2]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    path="../input/finetunedebertaadver/baseline_adv/Baseline_adv/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=175
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]

In [3]:
import os
import gc
import math
import time
import random
import scipy as sp
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from dataclasses import dataclass
from typing import Optional
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

from transformers import AdamW, AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true


import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Found existing installation: transformers 4.18.0
Uninstalling transformers-4.18.0:
  Successfully uninstalled transformers-4.18.0


Found existing installation: tokenizers 0.12.1
Uninstalling tokenizers-0.12.1:
  Successfully uninstalled tokenizers-0.12.1


Looking in links: ../input/pppm-pip-wheels-dataset
Processing /kaggle/input/pppm-pip-wheels-dataset/transformers-4.16.2-py3-none-any.whl
Processing /kaggle/input/pppm-pip-wheels-dataset/tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ../input/pppm-pip-wheels-dataset


tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


In [4]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [5]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


In [6]:
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
display(test.head())

,id,anchor,target,context,context_text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE


In [7]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

,id,anchor,target,context,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[SEP]altering gas flow[SEP]MECH...
2,36baf228038e314b,lower trunnion,lower locating,B60,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[SEP]lower locating[SEP]PERFORMI...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[SEP]upper portion[SEP]TEXTILES; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[SEP]artificial neural netwo...


In [8]:
# ====================================================
# tokenizer
# ====================================================
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

In [9]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

In [10]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [11]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [12]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=device)
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred1 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:01<00:00,  1.63it/s]


In [13]:
predictions = []
CFG.path = "../input/debertav3large/"
CFG.config_path = CFG.path+'config.pth'
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=device)
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred6 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:01<00:00,  1.64it/s]


In [14]:
class CFG:
    num_workers=4
    path="../input/debertabase/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-base"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=device)
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred7 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:00<00:00,  3.73it/s]


In [15]:
class CFG:
    num_workers=4
    path="../input/debertalarge5fold/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=device)
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred8 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:01<00:00,  1.93it/s]


In [16]:
class CFG:
    num_workers=0
    path="../input/electra/"
    config_path=path+'config.pth'
    model="google/electra-large-discriminator"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=5
    trn_fold=[0, 2, 4]

CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=device)
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred10 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:00<00:00,  3.97it/s]


In [17]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self.layer_norm1 = nn.LayerNorm(self.config.hidden_size)
        self._init_weights(self.attention)
        self.linear = nn.Linear(self.config.hidden_size, 1)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs[0]
        input_mask_expanded = inputs["attention_mask"].unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        out = sum_embeddings / sum_mask
        
        out = self.layer_norm1(out)
        output = self.fc(out)
        
        return output

In [18]:
class CFG:
    num_workers=4
    path="../input/debertalargelayernorm/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=device)
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
pred9 = np.mean(predictions, axis=0)

100%|██████████| 2/2 [00:01<00:00,  1.86it/s]


# 2.Roberta-large

In [19]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(seed=2019)

In [20]:
@dataclass(frozen=True)
class CFG:
    num_workers: Optional[int] = 4
    config_path: Optional[str] = '../input/robertalarge'
    model_path: Optional[str] = '../input/phrase-matching-roberta-training-pytorch-wandb'
    model_name: Optional[str] = 'roberta-large'
    batch_size: Optional[int] = 32
    max_len: Optional[int] = 128
    seed: Optional[int] = 2019
    num_targets: Optional[int] = 1
    n_folds: Optional[int] = 5
    tokenizer = AutoTokenizer.from_pretrained('../input/robertalarge')

In [21]:
PATH = '../input/us-patent-phrase-to-phrase-matching'
test = pd.read_csv(os.path.join(PATH, 'test.csv'))
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))

In [22]:
context_mapping = {
        "A": "Human Necessities",
        "B": "Operations and Transport",
        "C": "Chemistry and Metallurgy",
        "D": "Textiles",
        "E": "Fixed Constructions",
        "F": "Mechanical Engineering",
        "G": "Physics",
        "H": "Electricity",
        "Y": "Emerging Cross-Sectional Technologies",
}
    
test.context = test.context.apply(lambda x: context_mapping[x[0]])

In [23]:
class PhraseDataset:
    def __init__(self, anchor, target, context, tokenizer, max_len):
        self.anchor = anchor
        self.target = target
        self.context = context
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor)

    def __getitem__(self, item):
        anchor = self.anchor[item]
        context = self.context[item]
        target = self.target[item]

        encoded_text = CFG.tokenizer.encode_plus(
            context + " " + anchor,
            target,
            padding="max_length",
            max_length=self.max_len,
            truncation=True,
        )
        input_ids = encoded_text["input_ids"]
        attention_mask = encoded_text["attention_mask"]

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
        }

In [24]:
def inference_fn(model, test_loader):  
    model.eval()
    predictions = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    for data in tk0:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        with torch.no_grad():
            output = model(ids, mask)
        predictions.append(output.sigmoid().detach().cpu().numpy())
        
    return np.concatenate(predictions)

In [25]:
class PatentModel(torch.nn.Module):
    def __init__(self):
        super(PatentModel, self).__init__()
        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(CFG.config_path)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        
        self.transformer = AutoModel.from_pretrained(CFG.config_path, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(config.hidden_size, CFG.num_targets)
        
    def forward(self, ids, mask):
        transformer_out = self.transformer(input_ids=ids, attention_mask=mask)
        last_hidden_states = transformer_out[0]
        last_hidden_states = self.dropout(torch.mean(last_hidden_states, 1))
        logits1 = self.output(self.dropout1(last_hidden_states))
        logits2 = self.output(self.dropout2(last_hidden_states))
        logits3 = self.output(self.dropout3(last_hidden_states))
        logits4 = self.output(self.dropout4(last_hidden_states))
        logits5 = self.output(self.dropout5(last_hidden_states))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        
        return logits

In [26]:
def run_fold(test, fold, seed=42):    
    
    seed_everything(seed)
    
    test_dataset = PhraseDataset(
        test.anchor.values,
        test.target.values,
        test.context.values,
        CFG.tokenizer, 
        CFG.max_len
    ) 
    
    test_loader = DataLoader(test_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    model = PatentModel()
    
    model.load_state_dict(
        torch.load(f'{CFG.model_path}/{CFG.model_name.replace("-","_")}_patent_model_{fold}.pth',
        map_location=device
        )
    )
    
    model.to(device)

    preds = inference_fn(model, test_loader)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return preds

In [27]:
def inference_model(test, seed):
    
    predictions = []
    
    for f in range(CFG.n_folds):    
        preds = run_fold(test, f, seed) 
        predictions.append(preds)
        
    test_preds = np.column_stack(predictions)
        
    return test_preds

In [28]:
if __name__ == '__main__':
    pred2 =  np.mean(inference_model(test, CFG.seed),axis=1)

Some weights of the model checkpoint at ../input/robertalarge were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:00<00:00,  1.05it/s]
Some weights of the model checkpoint at ../input/robertalarge were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.de

*Bert for patent*

In [29]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import shutil

from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer

In [30]:
class CFG:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/bertforpatentsbaseline5folds/'
    
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 5
    batch_size = 64

In [31]:
test_df = pd.read_csv(f"{CFG.input_path}test.csv")
titles = pd.read_csv('../input/titles/titles.csv')
test_df = test_df.merge(test_df.merge(titles, left_on='context', right_on='code', sort=False))
test_df['input'] = test_df['title']+' '+test_df['anchor']

In [32]:
tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_path}uspppm_0')

In [33]:
class InferDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        
        return {
        **tokenizer( inputs, targets ),
        'label': -1
    }

In [34]:
from sklearn.preprocessing import MinMaxScaler

MMscaler = MinMaxScaler()

predictions = []

for fold in range(CFG.num_fold):
    te_dataset = InferDataset(test_df)
    model = AutoModelForSequenceClassification.from_pretrained(f'{CFG.model_path}uspppm_{fold}', num_labels=1)
    trainer = Trainer(
            model,
            tokenizer=tokenizer
        )

    outputs = trainer.predict(te_dataset)
    prediction = outputs.predictions.reshape(-1)
    
    predictions.append(prediction)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
pred3 = np.mean(predictions, axis=0)




***** Running Prediction *****
  Num examples = 36
  Batch size = 8


loading configuration file ../input/bertforpatentsbaseline5folds/uspppm_1/config.json
Model config BertConfig {
  "_name_or_path": "../input/bertforpatentsbaseline5folds/uspppm_1",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/bertforpatentsbaseline5folds/uspppm_1/pytorch_model.bin
All model checkpoint

loading configuration file ../input/bertforpatentsbaseline5folds/uspppm_2/config.json
Model config BertConfig {
  "_name_or_path": "../input/bertforpatentsbaseline5folds/uspppm_2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/bertforpatentsbaseline5folds/uspppm_2/pytorch_model.bin
All model checkpoint

loading configuration file ../input/bertforpatentsbaseline5folds/uspppm_3/config.json
Model config BertConfig {
  "_name_or_path": "../input/bertforpatentsbaseline5folds/uspppm_3",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/bertforpatentsbaseline5folds/uspppm_3/pytorch_model.bin
All model checkpoint

loading configuration file ../input/bertforpatentsbaseline5folds/uspppm_4/config.json
Model config BertConfig {
  "_name_or_path": "../input/bertforpatentsbaseline5folds/uspppm_4",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 39859
}

loading weights file ../input/bertforpatentsbaseline5folds/uspppm_4/pytorch_model.bin
All model checkpoint

In [35]:
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [36]:
class CFG_roberta:
    num_workers=4
    path="../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/"  ##change
    config_path=path+'config.pth'
    model="roberta-large"  ##change
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=175  ##change
    seed=99
    n_fold=4
    trn_fold=[0, 1, 2, 3]
class CFG_bertforpatent:
    num_workers=4
    path="../input/bertforpatent2roberta1/bert_for_patent2/f17f0b1e-dc40-11ec-a8f8-0242ac110002/"  ##change
    config_path=path+'config.pth'
    model="anferico/bert-for-patents"  ##change
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=115  ##change
    seed=99
    n_fold=4
    trn_fold=[0, 1, 2, 3]

In [37]:
import os
import gc
import math
import time
import random
import scipy as sp
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from dataclasses import dataclass
from typing import Optional
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

from transformers import AdamW, AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

        
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

    
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        self.fgm = FGM(self.model)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2


In [38]:
seed_everything(seed=42)
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
cpc_texts = torch.load("../input/cpc-texts/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
CFG_roberta.tokenizer = AutoTokenizer.from_pretrained(CFG_roberta.path+'tokenizer/')
test_dataset = TestDataset(CFG_roberta, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_roberta.batch_size,
                         shuffle=False,
                         num_workers=CFG_roberta.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_roberta.trn_fold:
    model = CustomModel(CFG_roberta, config_path=CFG_roberta.config_path, pretrained=False)
    state = torch.load(CFG_roberta.path+f"{CFG_roberta.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
torch.cuda.empty_cache()
pred4 = np.mean(predictions, axis=0)

Didn't find file ../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/tokenizer/added_tokens.json. We won't load it.
loading file ../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/tokenizer/vocab.json
loading file ../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/tokenizer/merges.txt
loading file ../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/tokenizer/tokenizer.json
loading file None
loading file ../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/tokenizer/special_tokens_map.json
loading file ../input/roberta6/63d73aa4-e891-11ec-9976-0242ac110002/tokenizer/tokenizer_config.json
100%|██████████| 2/2 [00:01<00:00,  1.67it/s]


In [39]:
seed_everything(seed=42)
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
cpc_texts = torch.load(CFG_bertforpatent.path+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
CFG_bertforpatent.tokenizer = AutoTokenizer.from_pretrained(CFG_bertforpatent.path+'tokenizer/')
test_dataset = TestDataset(CFG_bertforpatent, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_bertforpatent.batch_size,
                         shuffle=False,
                         num_workers=CFG_bertforpatent.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_bertforpatent.trn_fold:
    model = CustomModel(CFG_bertforpatent, config_path=CFG_bertforpatent.config_path, pretrained=False)
    state = torch.load(CFG_bertforpatent.path+f"{CFG_bertforpatent.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
del model, state, prediction; gc.collect()
torch.cuda.empty_cache()
pred5 = np.mean(predictions, axis=0)

Didn't find file ../input/bertforpatent2roberta1/bert_for_patent2/f17f0b1e-dc40-11ec-a8f8-0242ac110002/tokenizer/added_tokens.json. We won't load it.
loading file ../input/bertforpatent2roberta1/bert_for_patent2/f17f0b1e-dc40-11ec-a8f8-0242ac110002/tokenizer/vocab.txt
loading file ../input/bertforpatent2roberta1/bert_for_patent2/f17f0b1e-dc40-11ec-a8f8-0242ac110002/tokenizer/tokenizer.json
loading file None
loading file ../input/bertforpatent2roberta1/bert_for_patent2/f17f0b1e-dc40-11ec-a8f8-0242ac110002/tokenizer/special_tokens_map.json
loading file ../input/bertforpatent2roberta1/bert_for_patent2/f17f0b1e-dc40-11ec-a8f8-0242ac110002/tokenizer/tokenizer_config.json
100%|██████████| 2/2 [00:01<00:00,  1.64it/s]


In [40]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel

os.environ["WANDB_DISABLED"] = "true"


class CFG_DEB_SIMPLE:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/deberta-v3-large/deberta-v3-large'
    
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 4
    epochs = 5
    batch_size = 64
    max_input_length = 130
    batch_size = 64
    num_workers = 2

test_df = pd.read_csv(f"{CFG_DEB_SIMPLE.input_path}test.csv")
titles = pd.read_csv('../input/cpc-codes/titles.csv')
# test_df = test_df.merge(titles, left_on='context', right_on='code')
test_df = test_df.merge(test_df.merge(titles, left_on='context', right_on='code', sort=False))
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load("../input/folds-dump-the-two-paths-fix/cpc_texts.pth")
test_df['context_text'] = test_df['context'].map(cpc_texts)
test_df['text'] = test_df['anchor'] + '[SEP]' + test_df['target'] + '[SEP]'  + test_df['context_text']
test_df['text'] = test_df['text'].apply(str.lower)
display(test_df.head())

tokenizer_deberta_v3 = AutoTokenizer.from_pretrained(CFG_DEB_SIMPLE.model_path)

class Custom_Bert_Simple(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_path)
        config.num_labels = 1
        self.base = AutoModelForSequenceClassification.from_config(config=config)
        dim = config.hidden_size
        self.dropout = nn.Dropout(p=0)
        self.cls = nn.Linear(dim,1)
        
    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        base_output = self.base(input_ids=input_ids,
                                attention_mask=attention_mask,
                               token_type_ids=token_type_ids )

        output = base_output[0]
        if labels is None:
            return output
        
        else:
            return (nn.MSELoss()(torch.squeeze(output,1),labels), output)

class TestDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_length):
        self.text = df['text'].values.astype(str)
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        inputs = self.text[item]
        
        inputs = self.tokenizer(inputs,
                    max_length=self.max_input_length,
                    padding='max_length',
                    truncation=True )
        return torch.as_tensor(inputs['input_ids'], dtype=torch.long),\
               torch.as_tensor(inputs['token_type_ids'], dtype=torch.long),\
               torch.as_tensor(inputs['attention_mask'], dtype=torch.long)

def valid_fn(valid_loader, model, device):
    model.eval()
    preds = []
    labels = []
    for step, batch in enumerate(valid_loader):
        input_ids, token_type_ids, attention_mask = [i.to(device) for i in batch]
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask, token_type_ids)
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

predictions = []
MMscaler = MinMaxScaler()
te_dataset = TestDataset(test_df, tokenizer_deberta_v3, CFG_DEB_SIMPLE.max_input_length)
te_dataloader = DataLoader(te_dataset,
                              batch_size=CFG_DEB_SIMPLE.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG_DEB_SIMPLE.num_workers, pin_memory=True, drop_last=False)
for fold in tqdm(range(CFG_DEB_SIMPLE.num_fold)):
    
    model = Custom_Bert_Simple(CFG_DEB_SIMPLE.model_path)
    model.load_state_dict(torch.load('../input/us-patent-deberta-simple/microsoft_deberta-v3-large_best{}.pth'.format(fold))['model'])
    model.to('cuda')
    
    outputs = valid_fn(te_dataloader, model, 'cuda')
    prediction = outputs.reshape(-1)
    predictions.append(MMscaler.fit_transform(prediction.reshape(-1,1)).reshape(-1))

pred11 = np.mean(predictions, axis=0)

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,opc drum[sep]inorganic photoconductor drum[sep...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[sep]altering gas flow[sep]mech...
2,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[sep]lower locating[sep]performi...
3,1f37ead645e7f0c8,cap component,upper portion,D06,D06,TREATMENT OF TEXTILES OR THE LIKE; LAUNDERING;...,D,6.0,NaN,NaN,NaN,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[sep]upper portion[sep]textiles; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,H04,ELECTRIC COMMUNICATION TECHNIQUE,H,4.0,NaN,NaN,NaN,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[sep]artificial neural netwo...


loading configuration file ../input/deberta-v3-large/deberta-v3-large/config.json
Model config DebertaV2Config {
  "_name_or_path": "../input/deberta-v3-large/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.16.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

Didn't find file ../input/deberta-v3-large/deberta-v3-large/added_tokens.json. We wo

# Ensemble

In [41]:
w1 = 0.03# 0.8358  Deberta (public code)
w2 = 0.03# 0.8143  Roberta (public code)
w3 = 0.08# 0.826   BERT for patent (public code version 2)
w4 = 0.2# 0.8209  Roberta (jingyu)
w5 = 0.2# 0.8294  BERT for patent (jingyu)
w6 = 0.08# 0.8331  Deberta (gongzheng)   CV 0.860
w7 = 0.03# 0.8226  Deberta-base (gongzheng)  CV 0.852
w8 = 0.1# 0.8367   Deberta-5-fold (gongzheng)  CV 0.8625
w9 = 0.1# 0.8383   Deberta-layer-norm-5-fold (gongzheng) CV 0.8622
w10 = 0.2# 0.834    electra (gongzheng)
w11 = 0.2#  0.8392  Deberta  https://www.kaggle.com/code/surilee/inference-bert-for-uspatents-deepshare

w_list = [w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11]
w_list = [w / sum(w_list) for w in w_list]
w_list

[0.024, 0.024, 0.064, 0.16, 0.16, 0.064, 0.024, 0.08, 0.08, 0.16, 0.16]

In [42]:
from sklearn.preprocessing import MinMaxScaler

MMscaler = MinMaxScaler()

pred1_mm = MMscaler.fit_transform(pred1.reshape(-1,1)).reshape(-1)
pred2_mm = MMscaler.fit_transform(pred2.reshape(-1,1)).reshape(-1)
pred3_mm = MMscaler.fit_transform(pred3.reshape(-1,1)).reshape(-1)
pred4_mm = MMscaler.fit_transform(pred4.reshape(-1,1)).reshape(-1)
pred5_mm = MMscaler.fit_transform(pred5.reshape(-1,1)).reshape(-1)
pred6_mm = MMscaler.fit_transform(pred6.reshape(-1,1)).reshape(-1)
pred7_mm = MMscaler.fit_transform(pred7.reshape(-1,1)).reshape(-1)
pred8_mm = MMscaler.fit_transform(pred8.reshape(-1,1)).reshape(-1)
pred9_mm = MMscaler.fit_transform(pred9.reshape(-1,1)).reshape(-1)
pred10_mm = MMscaler.fit_transform(pred10.reshape(-1,1)).reshape(-1)
pred11_mm = MMscaler.fit_transform(pred11.reshape(-1,1)).reshape(-1)

final_predictions =  pred1_mm * w_list[0] + pred2_mm * w_list[1] + pred3_mm * w_list[2] + pred4_mm * w_list[3] + pred5_mm * w_list[4] + pred6_mm * w_list[5] + pred7_mm * w_list[6] + pred8_mm * w_list[7] + pred9_mm * w_list[8] + pred10_mm * w_list[9] + pred11_mm * w_list[10]
# final_predictions = pred11_mm

In [43]:
# pred_samples = list(zip(pred1_mm, pred2_mm, pred3_mm, pred4_mm, pred5_mm, pred6_mm, pred7_mm, pred8_mm, pred9_mm, pred10_mm, pred11_mm))

# final_predictions = []

# for pred_sample in pred_samples:
#     max_pred, min_pred = max(pred_sample), min(pred_sample)
#     max_pred_id, min_pred_id = pred_sample.index(max_pred), pred_sample.index(min_pred)
    
#     pred_w_list = w_list.copy()
#     pred_w_list[max_pred_id] = 0
#     pred_w_list[min_pred_id] = 0
    
#     prediction = 0
#     for each_model_predict, each_w in zip(pred_sample, pred_w_list):
#         prediction += each_model_predict * each_w
    
#     prediction = prediction / sum(pred_w_list)
    
# #     prediction = (sum(pred_sample) - max(pred_sample) - min(pred_sample)) / (len(pred_sample) - 2)
#     final_predictions.append(prediction)

In [44]:
# pred_samples = list(zip(pred1_mm, pred2_mm, pred3_mm, pred4_mm, pred5_mm, pred6_mm, pred7_mm, pred8_mm, pred9_mm, pred10_mm, pred11_mm))

# final_predictions = []

# for pred_sample in pred_samples:
#     prediction = (sum(pred_sample) - max(pred_sample) - min(pred_sample)) / (len(pred_sample) - 2)
#     final_predictions.append(prediction)

In [45]:
def _upd_score_between(data, thresholds, value):
    """\o/"""
    mask_th = data.between(*thresholds, inclusive='both')
    data[mask_th] = value


def upd_score(data, th_dict=None):
    """\o/"""
    if isinstance(data, pd.Series):
        result = data.copy()
    else:
        return data

    if not th_dict:        
        th_dict = {
            '0': 0.02,
            '.25': (0.24, 0.26),
            '.50': (0.49, 0.51),
            '.75': (0.74, 0.76),
            '1': 0.98
        }

    if isinstance(th_dict, dict):    
        th0 = th_dict.get('0')
        th25 = th_dict.get('.25')
        th50 = th_dict.get('.50')
        th75 = th_dict.get('.75')
        th100 = th_dict.get('1')
    else:
        return data
    
    if th0:
        if isinstance(th0, float):
            th0 = (result.min(), th0)
        
        if isinstance(th0, tuple):
            _upd_score_between(result, th0, 0)
    
    if th25 and isinstance(th25, tuple):
        _upd_score_between(result, th25, 0.25)

    if th50 and isinstance(th50, tuple):
        _upd_score_between(result, th50, 0.50)
            
    if th75 and isinstance(th75, tuple):
        _upd_score_between(result, th75, 0.75)
            
    if th100:
        if isinstance(th100, float):
            th100 = (th100, result.max())
        
        if isinstance(th100, tuple):
            _upd_score_between(result, th100, 1)

    return result


thresholds_dict = {
    '0': 0.02,
    '.25': (0.24, 0.26),
    '.50': (0.49, 0.51),
    '.75': (0.74, 0.76),
    '1': 0.98
}


sub['score'] = final_predictions
sub['score'] = upd_score(sub['score'], thresholds_dict)
sub.to_csv('submission.csv', index=False)

# Submission

In [46]:
sub.head()

,id,score
0,4112d61851461f60,0.513264
1,09e418c93a776564,0.692246
2,36baf228038e314b,0.468795
3,1f37ead645e7f0c8,0.266728
4,71a5b6ad068d531f,0.000000


In [47]:
sub['pred1'] = pred1_mm
sub['pred2'] = pred2_mm
sub['pred3'] = pred3_mm
sub['pred4'] = pred4_mm
sub['pred5'] = pred5_mm
sub['pred6'] = pred6_mm
sub['pred7'] = pred7_mm
sub['pred8'] = pred8_mm
sub['pred9'] = pred9_mm
sub['pred10'] = pred10_mm
sub['pred11'] = pred11_mm

In [48]:
sub.head(20)

,id,score,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,pred10,pred11
0,4112d61851461f60,0.513264,0.548070,0.582516,0.470140,0.523272,0.542626,0.412257,0.599047,0.574789,0.556047,0.638238,0.325940
1,09e418c93a776564,0.692246,0.722803,0.741248,0.650935,0.733734,0.624701,0.707529,0.712458,0.750050,0.728954,0.758063,0.600674
2,36baf228038e314b,0.468795,0.435978,0.407745,0.433883,0.479464,0.429251,0.473934,0.511296,0.526621,0.501007,0.426838,0.514221
3,1f37ead645e7f0c8,0.266728,0.290185,0.287422,0.254011,0.258861,0.242358,0.253762,0.244287,0.234795,0.235337,0.316446,0.287925
4,71a5b6ad068d531f,0.000000,0.013673,0.026354,0.000000,0.000095,0.000130,0.000105,0.000916,0.004200,0.002166,0.002237,0.112711
5,474c874d0c07bd21,0.500000,0.467670,0.495716,0.471766,0.488659,0.498857,0.466096,0.512878,0.480865,0.478713,0.579902,0.499533
6,442c114ed5c4e3c9,0.418388,0.468271,0.486066,0.367578,0.508660,0.340060,0.475942,0.475008,0.466500,0.447965,0.297359,0.459802
7,b8ae62ea5e1d8bdb,0.000000,0.000000,0.000130,0.002789,0.000023,0.000004,0.000000,0.000000,0.000000,0.000015,0.000000,0.002243
8,faaddaf8fcba8a3f,0.288120,0.300080,0.294383,0.238507,0.283252,0.238643,0.283237,0.326393,0.311302,0.320707,0.362896,0.253126
9,ae0262c02566d2ce,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
